In [1]:
import helper_fxns as hf
import importlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from pylab import get_cmap
import requests
import sys

In [7]:
%gui qt

In [79]:
importlib.reload(hf)

<module 'helper_fxns' from '/Users/clintonwang/Documents/Work/LI-RADS/helper_fxns.py'>

In [2]:
img,_ = hf.dcm_load('./dcm1')

In [40]:
img1 = img[:,:,20:25]
img1 = np.array([list(img1), list(img1), list(img1)])
img1 = np.transpose(img1, (1,2,3,0))

In [41]:
np.save('img3', img1)

# Overall approach
This is a tool for classifying precontrast and 20s postcontrast MRIs (into LI-RADS stages / predicting survival?).

#### Training
-In mini-batches:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
5. For HCC labels, calculate and store embeddings (feature values)
6. Train CNN on all

-After CNN is trained, then:
7. Train feature engineered network on HCC labels

#### Test pipeline:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Use CNN to classify as HCC/non-HCC
5. If HCC, calculate features values
6. If HCC, use feature engineered network to predict label

#### CNN models
First, try VGGNet, then ResNet. Then try VAE-GAN, training on benign only; deviation score = malignancy.

In [ ]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [ ]:
def training_pipeline(pre_dcm_fns, post_dcm_fns, labels):
    """
    pre_dcm_fn is the precontrast image, and post_dcm_fn is the postcontrast image
    1. Import pre/20s DICOMs
    2. Register pre to 20s
    3. Segment whole liver
    4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
    5. For HCC labels, calculate and store embeddings (feature values)
    6. Train CNN on all
    7. Train feature engineered network on HCC labels
    """
    train_frac = 0.7
    data_size = len(pre_dcm_fns)
    
    # TODO: do a proper k-fold x-validation split
    X_cnn_train = X_cnn[:data_size*train_frac]
    X_cnn_val = X_cnn[data_size*train_frac:]
    
    for x in range(data_size):
        pre_dcm_fn = pre_dcm_fns[x]
        post_dcm_fn = post_dcm_fn[x]
        label = labels[x]
        
        pre_img = hf.dcm_load(pre_dcm_fn)
        post_img = hf.dcm_load(post_dcm_fn)

        pre_img = seg_liver(pre_img)
        post_img = seg_liver(post_img)
        
        y_cnn = label['is-hcc']
        label.pop('is-hcc')
        X_cnn = [pre_img, post_img, label]
        
        """if y_cnn:
            features = get_features(pre_img, post_img, label)
            
            # rfm is random forest model
            y_rfm = label
            X_rfm = [pre_img, post_img, label]"""

In [ ]:
def dcm_2_ni(fn):
    pass

In [ ]:
def reg_dcms(moving, target):
    pass

def seg_liver(img):
    pass